In [145]:
clauses = []

num_literal = 0
num_clauses = 0

with open('SAT2018/Biere/sdiv15prop.cnf') as file:
    for line in file:
        if(line[0] == 'p'):
            l = line.split()
            num_literal = int(l[2])
            num_clauses = int(l[3])
        if(line[0] != 'p' and line[0] != '%' and line[0] != 'c'):
            clause = [int(digit) for digit in line.split()]
            clauses.append(clause[:-1])

In [146]:
literal_clauses = [[] for _ in range(num_literal)]

In [147]:
for i in range(len(clauses)):
    for literal in clauses[i]:
        if literal > 0:
            literal_clauses[abs(literal)-1].append(i+1)
        else:
            literal_clauses[abs(literal)-1].append(-(i+1))

In [148]:
first_moment = [0.5] * num_literal
second_moment = [0.375] * num_literal

In [149]:
def update_message(literal, clause):
    input_clause = clauses[abs(clause)-1]
    
    length_clause = len(input_clause)
    
    p = 1.0
    
    for i in range(length_clause):
        lit = input_clause[i]
        if(lit > 0):
            p = p * (1 - first_moment[abs(lit)-1])
        else:
            p = p * first_moment[abs(lit)-1]
    #print(p)
            
    M1 = first_moment[literal]
    M2 = second_moment[literal]
    
    left_extre = 0.0001
    right_extre = 0.9999
    if(M1 < left_extre or M1 > right_extre):
        return
    
            
    if clause > 0:
        p_omit = p / (1 - M1)
        first_moment[literal] =  (M1 - p_omit * (M1 - M2))/ (1 - p)
        left = first_moment[literal]**2
        right = first_moment[literal]
        second_moment[literal] = (left + right)/2
    else:
        p_omit = p / M1
        first_moment[literal] = (M1 - p_omit * M2) / (1 - p)
        left = first_moment[literal]**2
        right = first_moment[literal]
        second_moment[literal] = (left + right)/2

In [150]:
def update():
    for literal in range(num_literal):
        for clause in literal_clauses[literal]:
            update_message(literal, clause)

In [151]:
def main(epochs):
    for i in range(epochs):
        update()

In [152]:
main(1000)

In [153]:
def satisfied_clauses():
    polarity = [0] * num_literal
    for k in range(num_literal):
        if(first_moment[k] > 0.5):
            polarity[k] = 1
        else:
            polarity[k] = -1
    num_satisfied = 0
    for k in range(num_clauses):
        clause = clauses[k]
        for l in clause:
            if(l * polarity[abs(l) - 1] > 0):
                num_satisfied += 1
                break
    return num_satisfied

In [154]:
satisfied_clauses()

8994

In [143]:
num_clauses

9236